In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [3]:
mol_type = "6r7m"
algo = "rwm"
energy = "ma"
folder = "../../Data/hpc_out/$(algo)_$(energy)_2_$(mol_type)/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end

            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")
for elem in evaluation_strings
    println(elem)
end

Minimal configuration id: 65 | Average acceptance rate: 0.3759299311573066
1: 3.1 | 1.1 | 0.28492713 | 8762.74153046811, 8707.574525372189, 8726.768578786901 | 29.08773017762365
2: 3.1 | 1.1 | 0.93681175 | 8762.741529905985, 8732.980612100057, 8762.741530252486 | 16.951075579821048
3: 3.1 | 1.1 | 0.28328994 | 8762.741530039755, 8713.408071976446, 8755.187476529953 | 15.970545999294254
4: 3.1 | 1.1 | 0.34266594 | 8762.741530194533, 8695.772593553918, 8700.026115617713 | 5.750536094536069
5: 3.1 | 1.1 | 0.071613126 | 8762.74153041354, 8699.444548768908, 8711.627748292109 | 21.138186529624797
6: 3.1 | 1.1 | 0.114833534 | 8762.741530096444, 8697.280316489172, 8697.280316489172 | 8.386456438699236

8: 3.1 | 1.1 | 0.30563742 | 8762.741529776355, 8682.152950560685, 8692.862175442375 | 1.3270045408889264
9: 3.1 | 1.1 | 0.075455815 | 8762.741530534793, 8700.35189012054, 8703.013262421613 | 15.978857152054873
10: 3.1 | 1.1 | 0.5752232 | 8762.741529861714, 8713.049630274212, 8731.970630071382 | 1

In [22]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [ ]:
@load "$(folder)/11_$(algo_e)_2_$(mol_type).jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [15]:
"$(folder)/11_$(algo_e)_2_$(mol_type).jld2"

"../../Data/hpc_out/2_6r7m/rwm_ma_2_6r7m//11_rwm_ma_2_6r7m.jld2"

In [13]:
using GeometryBasics

In [ ]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
